In [ ]:
import zipfile
import os
zip_path = '/content/archive.zip'
extract_path = 'dataset_raw'
os.makedirs(extract_path,exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print('SUCCESS')

SUCCESS


In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

xml_dir = "dataset_raw/annotations"
img_dir = "dataset_raw/images"

output_img_dir = "dataset/images"
output_label_dir = "dataset/labels"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

classes = {"car": 0}

def convert_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find("filename").text
    img_path = os.path.join(img_dir, filename)

    img = Image.open(img_path)
    w, h = img.size

    yolo_labels = []

    for obj in root.iter("object"):
        cls = obj.find("name").text.lower()
        if cls not in classes:
            continue

        xmlbox = obj.find("bndbox")
        xmin = int(xmlbox.find("xmin").text)
        xmax = int(xmlbox.find("xmax").text)
        ymin = int(xmlbox.find("ymin").text)
        ymax = int(xmlbox.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        width = (xmax - xmin) / w
        height = (ymax - ymin) / h

        yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")

    return filename, yolo_labels

for xml in os.listdir(xml_dir):
    filename, labels = convert_xml(os.path.join(xml_dir, xml))

    with open(os.path.join(output_label_dir, filename.replace(".png", ".txt")), "w") as f:
        f.write("\n".join(labels))

    os.system(f"cp {os.path.join(img_dir, filename)} {output_img_dir}")

print("Converted to YOLO format successfully!")

Converted to YOLO format successfully!


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

## create data.yaml
yaml_cotent = """train: /content/dataset/images
val: /content/dataset/images

nc: 1
names: ['car']
"""

with open("data.yaml", "w") as f:
    f.write(yaml_cotent)

model = YOLO("yolov8n.pt")

model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=16
)

runs/detect/train/weights/best.pt

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

KeyboardInterrupt: 

In [ ]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 16.7 MB/s eta 0:00:00


In [ ]:
import cv2
import easyocr
from ultralytics import YOLO

reader = easyocr.Reader(['en'])
model = YOLO("best.pt")

def detect_plate(image_path):
    results = model(image_path)
    img = cv2.imread(image_path)

    detected_texts = []

    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        crop = img[y1:y2, x1:x2]

        result = reader.readtext(crop)
        if result:
            detected_texts.append(result[0][-2])

    return detected_texts


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'

In [ ]:
pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import sqlite3
from datetime import datetime

conn = sqlite3.connect("vehicles.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS vehicle_logs (
    plate TEXT,
    timestamp TEXT
)
""")

def insert_plate(plate):
    cursor.execute(
        "INSERT INTO vehicle_logs VALUES (?, ?)",
        (plate, datetime.now())
    )
    conn.commit()